In [1]:
from Data_Loaders import loaders
from pathlib import Path
BASE_DIR = Path("/home/nbiescas/Desktop/CVC/CVC_internship") #or Path().absolute()
DATA_PATH = BASE_DIR / "omniglot.npz"

In [2]:
import torch
from dgl.nn.pytorch import GraphConv
from torch import nn
from torch.nn.parameter import Parameter
import torch.nn.functional as F

device = 'cuda'
class VGAEModel(nn.Module):
    def __init__(self, in_dim, hidden1_dim, hidden2_dim):
        super(VGAEModel, self).__init__()
        self.in_dim = in_dim
        self.hidden1_dim = hidden1_dim
        self.hidden2_dim = hidden2_dim

        layers = [GraphConv(self.in_dim, self.hidden1_dim, activation=F.sigmoid, allow_zero_in_degree=True),
                  GraphConv(self.in_dim, self.hidden1_dim, activation=F.relu, allow_zero_in_degree=True),
                  GraphConv(self.in_dim, self.hidden1_dim, activation=F.relu, allow_zero_in_degree=True),
                  GraphConv(self.in_dim, self.hidden1_dim, activation=F.relu, allow_zero_in_degree=True),
                  GraphConv(self.in_dim, self.hidden1_dim, activation=F.relu, allow_zero_in_degree=True),
                  GraphConv(self.in_dim, self.hidden1_dim, activation=F.relu, allow_zero_in_degree=True),
                  GraphConv(self.in_dim, self.hidden1_dim, activation=F.relu, allow_zero_in_degree=True),
                  GraphConv(self.hidden1_dim, self.hidden2_dim, activation=lambda x: x, allow_zero_in_degree=True),
                  GraphConv(self.hidden1_dim, self.hidden2_dim, activation=lambda x: x, allow_zero_in_degree=True)]
        self.layers = nn.ModuleList(layers)

    def encoder(self, g, features):
        print(g)
        for layer in self.layers[:-2]:
            h = layer(g, features)
        self.mean = self.layers[-2](g, h)
        self.log_std = self.layers[-1](g, h)
        gaussian_noise = torch.randn(features.size(0), self.hidden2_dim).to(device)
        sampled_z = self.mean + gaussian_noise * torch.exp(self.log_std.mul(0.5)).to(device) #The think that changes is that he is using the log_std insted of the log_var
        return sampled_z

    def decoder(self, z):
        adj_rec = torch.sigmoid(torch.matmul(z, z.t()))
        return adj_rec

    def forward(self, g, features):
        z = self.encoder(g, features)
        adj_rec = self.decoder(z)
        return adj_rec

In [3]:
train, val, test = loaders(DATA_PATH)
model = VGAEModel(2, 10, 15).to('cuda')
graph, adj = next(iter(train))

In [26]:
train.batch_size


128

In [7]:
feat = graph.ndata.pop('feat')

In [10]:
feat

tensor([[0.5000, 0.5000],
        [0.5046, 0.4954],
        [0.5446, 0.4554],
        ...,
        [0.6421, 0.3579],
        [0.6354, 0.3646],
        [0.6870, 0.3130]])

In [27]:
graph.batch_num_nodes()

tensor([ 36,  28,  28,  26,  38,  37,  29,  38,  48,  20,  27,  32,  37,  43,
         28,  27,  11,  28,  30,  28,  21,  22,   5,  45,  40,  60,   6,  18,
         23,  31,  37,  20,  36,  26,  23,  32,  25,  13,  44,  52,  22,  21,
         34,   7,  20,  15,  17,   4,  44,  21,  35,  13,  45,  28,  27,  57,
         18,  31,  20,  47,  68,  18,  17,  30,  58,  38,  15,  38,  42,  23,
          7,  25,  22,  31,  14,  19,  43,  16,  22,  24,  22,  17,  29,  33,
         11,  56,   7,  28,  50,   8,  30,  15,  29,  24,  55,  34,  27,  25,
         32,  32,  44,   9,  40,  21,  45,  14,  25,  35,  28,  43, 109,  10,
         20,  26,  23,  26,  13,  35,  45,  34,  49,  21,  38,  12,  13,  54,
         27,   6])

In [39]:
a = torch.tensor([[2, 3],
               [6, 3],
               [1, 3],
               [9, 3],
               [43, 3],
               [23, 3],
               [54, 3],
               [2, 3]], dtype=torch.float)

In [42]:
mean = a.mean(dim=0)
std = a.std(dim=0)

In [44]:
nor = (a - mean) / std

In [45]:
nor

tensor([[-0.7525,     nan],
        [-0.5583,     nan],
        [-0.8010,     nan],
        [-0.4127,     nan],
        [ 1.2380,     nan],
        [ 0.2670,     nan],
        [ 1.7720,     nan],
        [-0.7525,     nan]])

In [22]:
start_idx = 0
output_matrix = []
for nodes in graph.batch_num_nodes():
    end_index = start_idx + nodes
    features_graph = feat[start_idx:end_index]
    output_matrix.append(torch.sigmoid(torch.matmul(features_graph, features_graph.t())))
    start_idx = end_index

In [24]:
output_matrix

[tensor([[0.6225, 0.6225, 0.6225,  ..., 0.6225, 0.6225, 0.6225],
         [0.6225, 0.6225, 0.6226,  ..., 0.6225, 0.6225, 0.6225],
         [0.6225, 0.6226, 0.6234,  ..., 0.6225, 0.6224, 0.6225],
         ...,
         [0.6225, 0.6225, 0.6225,  ..., 0.6225, 0.6225, 0.6225],
         [0.6225, 0.6225, 0.6224,  ..., 0.6225, 0.6225, 0.6225],
         [0.6225, 0.6225, 0.6225,  ..., 0.6225, 0.6225, 0.6225]]),
 tensor([[0.6225, 0.6225, 0.6225, 0.6225, 0.6225, 0.6225, 0.6225, 0.6225, 0.6225,
          0.6225, 0.6225, 0.6225, 0.6225, 0.6225, 0.6225, 0.6225, 0.6225, 0.6225,
          0.6225, 0.6225, 0.6225, 0.6225, 0.6225, 0.6225, 0.6225, 0.6225, 0.6225,
          0.6225],
         [0.6225, 0.6230, 0.6230, 0.6231, 0.6225, 0.6224, 0.6220, 0.6212, 0.6211,
          0.6151, 0.6147, 0.6146, 0.6151, 0.6151, 0.6155, 0.6160, 0.6163, 0.6174,
          0.6176, 0.6182, 0.6203, 0.6175, 0.6158, 0.6155, 0.6141, 0.6141, 0.6144,
          0.6144],
         [0.6225, 0.6230, 0.6230, 0.6231, 0.6225, 0.6224, 0.6220

In [59]:
train, val, test = loaders(DATA_PATH)
model = VGAEModel(2, 10, 15).to('cuda')
graph, adj = next(iter(train))
output = model(graph.to('cuda'), graph.ndata['feat'].float().to('cuda'))

Graph(num_nodes=281, num_edges=648,
      ndata_schemes={'feat': Scheme(shape=(2,), dtype=torch.float64)}
      edata_schemes={})


In [70]:
adj.size()

torch.Size([10, 5329])

torch.Size([78961])

In [64]:
graph.adj().to_dense()

tensor([[1., 1., 0.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [0., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 1., 0.],
        [0., 0., 0.,  ..., 1., 1., 1.],
        [0., 0., 0.,  ..., 0., 1., 1.]])